In [36]:
import numpy as np
import quantecon as qe
import matplotlib.pyplot as plt
import time
from scipy.optimize import minimize_scalar
from scipy.interpolate import interp1d, InterpolatedUnivariateSpline

Create grids for next period's capital holdings

In [4]:
def create_grids(n, theta):

    zero_grid=np.linspace(0, 1, n).reshape(-1,1)

    k_grid=np.zeros(shape=(n,1))

    k_low  = 0.01

    k_high  = 2
    
    #for i in range(len(new_grid)):
        #k_grid[i] =  low_bound + ( up_bound - low_bound ) * (new_grid[i] ** 1)

    for i in range(len(zero_grid)):
        k_grid[i] =  (0.25 * k_low) + ( (1.5 * k_high) - (0.25 * k_low) ) * (zero_grid[i] ** theta)

    return k_grid 

define various utility functions

In [5]:
def u(c, gamma):
    if gamma == 1:
        return np.log(c)
    else:
        return ((c ** (1 - gamma))/( 1 - gamma))

In [6]:
def u_ces(c,gamma, beta):
    return (1 - beta) * (c) ** (1-gamma)

define the production function

In [7]:
def f(A, alpha, delta, k):
    return A * (k ** alpha) + (1 - delta) * k

Now, create a function to linearly interpolate each column of the Value matrix, so that we can calculate the derivate at each point on the capital grid.

In [38]:
def interpolate_column(grid, column):
    interpolator = interp1d(np.squeeze(grid), column, kind = 'linear', fill_value = 'extrapolate')
    return interpolator

def 

Let's now begin with the endogenous grid method

In [35]:
k_grid = create_grids(101, 3)
# V = np.zeros((101, 15))

# for i in range(len(V[0])):
#    for j in range(len(V[0])):
#        V[i,j]=(0.01*(i+1))*(j+1)

i_vals = np.arange(1,102)
j_vals = np.arange(1,16)

V = (0.01 * (k_grid)) * j_vals  

# print(V.shape)

#for j in range(len(V[1])):
#   print(f'column {j}')
#  print(V[:,j])

#for j in range(len(V[1])):
#
#    fig, ax = plt.subplots(figsize=(10, 6))
#
#    for j in range(len(V[1])):
#        ax.plot(k_grid, V[:, j], label=f'TFP Shock {j}')

#    ax.set_title('Value Function')
#    ax.set_xlabel('Capital Stock')
#    ax.set_ylabel('Value')
#    ax.legend()
#    plt.show()


In [71]:
def egm(n_k, n_z, epsilon, theta, rho, sigma, mu, toler, beta, gamma, max_iter):
    # First, discretise an AR1 process using Rouwenhorst's method
    mc = qe.markov.approximation.rouwenhorst(n_z, rho, sigma, mu)
    z, Pi = mc.state_values, mc.P

    # Create capital, shock and resource grids
    z_grid = z
    k_grid = create_grids(n_k, theta)
    Y_grid = np.meshgrid(k_grid, z_grid, indexing='ij')

    #Create a matrix to store the derivatives of the interpolated value functions on the capital grid
    derivatives = np.array((n_k,n_z))

    # Initialise value and policy functions
    j_vals = np.arange(1,16)

    V_star = (0.01 * (k_grid)) * j_vals

    V_star_new = np.zeros((n_k, n_z))   

    interpolated_V = np.zeros((n_k,n_z))

    # G_kp = np.zeros((n_k, n_z))

    # G_kp_old = np.copy(G_kp)

    it = 0

    for it in range(max_iter):

        # Create list of interpolated columns
        # interpolators = []
        # for j in range(n_z)
        interpolators = [interpolate_column(k_grid, V_star[:,j]) for j in range(n_z)]

        # Update the derivatives

        for j in interpolators:
            for i in range(n_k):
                #for j in range(len(n_z)):
                    derivatives[i,j]=((interpolators[j](k_grid[i])-interpolators[j](k_grid[i]-epsilon))/(epsilon))

        # Now that we have the derivatives, we can use the first-order conditions to find consumption

        c_star = (derivatives)**(-1/gamma)

        # Calculate the implied total resources

        Y_star = c_star + k_grid

        # Update the Value function

        V = u(c_star) + V_star

        # New interpolation 

        V_interpolators = [interpolate_column(Y_star[:,j], V_star[:,j]) for j in range(n_z)]

        # Evaluate 

        for j in V_interpolators:
            for i in range(n_k):
                interpolated_V[i,j] = V_interpolators[j](Y_grid[i, j])

        # Now, calculate the updated expected value

        for j in range(n_z):
            V_star_new[:, j] = beta * np.dot(interpolated_V[:, j], Pi[j,:])
        
        # Check error

        error = np.max(np.abs(V_star_new - V_star)/1+np.abs(V_star))

        if it == 0:
            print(f'Iteration {it} completed with error = {error}')

        if it % 25 == 0:
            print(f'Iteration {it} completed with error = {error}')

        if error < toler:
            break

        V_star = np.copy(V_star_new)
        
        it+=1

    return V_star

In [72]:
v = egm(n_k=1001, n_z=15, epsilon=1e-6, theta=2, rho=0.98, sigma=0.02, mu=0, toler=4e-7, beta=0.9, gamma=2, max_iter=10000)

C:\Users\bayle\AppData\Local\Temp\ipykernel_29376\2225383842.py:3: UserWarning: The API of rouwenhorst has changed from `rouwenhorst(n, ybar, sigma, rho)` to `rouwenhorst(n, rho, sigma, mu=0.)`. To find more details please visit: https://github.com/QuantEcon/QuantEcon.py/issues/663.
  mc = qe.markov.approximation.rouwenhorst(n_z, rho, sigma, mu)


TypeError: list indices must be integers or slices, not interp1d